In [2]:
import sys
import glob
import numpy as np
import astropy.units as u
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import matplotlib
from matplotlib.colors import SymLogNorm
from astropy.cosmology import Planck15 as cosmo
from astropy import constants as const
from powerbox import get_power
from hmf import MassFunction
from xcorr import load_binary_data
import sys
import os
import numpy as np
from scipy.ndimage.filters import gaussian_filter
import xcorr

plt.rc('font', family='serif')

In [3]:
def H(z):
    '''
    Returns astropy Hubble constant at given redshift
    
    Units: km Mpc^-1 s^-1
    '''
    return cosmo.H(z)

def n_rec_dot(T_k, x, delta_x, z):
    """
    """
    return alpha(T_k, z) * n_e(x, delta_x, z) * n_HII(x, delta_x, z)

def n_e(x, delta_x, z):
    """
    """
    return x * n_b(delta_x, z)

def n_HII(x, delta_x, z, Y_He = 0.24):
    """
    """
    return n_e(x, delta_x, z) * (4. - 4. * Y_He) / (4. - 3 * Y_He)

def n_b(delta_x, z, n_b0 = 1.905e-7 * u.cm ** -3):
    """
    """
    return n_b0 * (1. + delta_x) #* (1 + z) ** 3


def alpha(T_k, z):
    """
    Recombination coefficient
    """
    units = u.cm ** 3 / u.s
    return 4.2e-13 * (T_k / 2e4) ** -0.7 * (1 + z) ** 3 * units

def L_diffuse(T_k, x, delta_x, z, f_rec = 0.66):
    """
    """
    E_lya = 1.637e-11 * u.erg
    return f_rec * n_rec_dot(T_k, x, delta_x, z) * E_lya

def I_diffuse(T_k, x, delta_x, z):
    """
    """
    c = y(z) * cosmo.comoving_transverse_distance(z) ** 2 / (4 * np.pi * cosmo.luminosity_distance(z) ** 2)
    nu = 2.47e15 / u.s / (1 + z)
    return (L_diffuse(T_k, x, delta_x, z) * c * nu).to(u.erg / u.cm ** 2 / u.s)

def y(z):
    '''
    wl_lya -> Lyman-alpha wavelength in units of km
    
    Returns value in units of Mpc s
    '''
    l_lya = 1.215e-7 * u.m 
    return l_lya * (1.0 + z) ** 2 / H(z)

In [4]:
ir_cubes = glob.glob('../../data/lyman_alpha_cube_*npy')
ir_cubes.sort()

In [5]:
density = glob.glob('../../data/updated_smoothed_deltax_z*')
density.sort()

neutral_frac = glob.glob('../../data/xH_z*')
neutral_frac.sort()

In [10]:
def z_val(string):
    """
    """
    return float(os.path.basename(string).split('_')[-1].rsplit('.', 1)[0])

def z_str(string):
    """
    """
    return os.path.basename(string).split('_')[-1].rsplit('.', 1)[0].replace('.', '_')

In [16]:
for i in np.arange(len(density)):
    delta_x = load_binary_data(density[i])
    delta_x = np.reshape(delta_x, (256, 256, 256))
    
    x_i = load_binary_data(neutral_frac[i])
    x_i = np.reshape(x_i, (256, 256, 256))
    
    diff_cube = I_diffuse(5e3, 1.0 - x_i, delta_x, z = z_val(ir_cubes[i]))
    print (os.path.join('../../data', 'diffuse_ir_cube_' + z_str(ir_cubes[i]) + '.npy'))
    np.save(os.path.join('../../data', 'diffuse_ir_cube_' + z_str(ir_cubes[i]) + '.npy'), diff_cube.value)

../../data/diffuse_ir_cube_6_0.npy
../../data/diffuse_ir_cube_6_14.npy
../../data/diffuse_ir_cube_6_28.npy
../../data/diffuse_ir_cube_6_43.npy
../../data/diffuse_ir_cube_6_58.npy
../../data/diffuse_ir_cube_6_73.npy
../../data/diffuse_ir_cube_6_88.npy
../../data/diffuse_ir_cube_7_04.npy
../../data/diffuse_ir_cube_7_2.npy
../../data/diffuse_ir_cube_7_37.npy
../../data/diffuse_ir_cube_7_53.npy
../../data/diffuse_ir_cube_7_7.npy
../../data/diffuse_ir_cube_7_88.npy
../../data/diffuse_ir_cube_8_06.npy
